# Avalanche Risk Prediction
### Jaymin West
### Spring, 2023

The orginal idea for this project was to create a model that couuld predict the layers within a snow pack in a given region. Initially, I thought that I would be able to use weather data to achive this goal. However, it became evident that the data I was hoping to use was too sparse and what I thought was a datascience problem was actually a meteorology problem. As such, I had to adjust the scope of this project to focusing more on predicting the overall avalanche risk in a given area. 

### Step 1: Gathering The Data

The area I chose to use was that covered by the Northwest Avalanche Center (NWAC). NWAC covers the western half of Washington and into Oregon. NWAC is a small part of Avalanche.org who host the websites for dozens of avalnche centers around the country. This is important as I built a webscraper to scrape all the archival data from NWAC's site and that webscraper could be expaned to work on all of the other avalanche centers. The code I wrote for the webscraper can be found in the webscraper.py file. It goes through a table of dates and gets the corresponding risk, area, and forecast for each date. The forecast comes in three sections, all of which go into varying levels of detail about the observed conditions and risks in the area

In [1]:
import webscraper

# Getting the avalanche data at Snoqualmie Pass for the current season: 
ws = webscraper.Webscraper("Snoqualmie Pass")
ws.open_archive_page()
reports_data = ws.scrape_daily_reports()
ws.to_csv(reports_data)

KeyboardInterrupt: 

### Step 2: Processing The Data

I was not expecting this to be one of the more time consuming section of the project but it was. It took me a long time trying to figure out how to best use my text data to predict the avalanche risk in the future. I figured I would have to do a lot of preprocessing to get my text into a state that would be useful for the computer. I took all of the text data, removed the stop words (including some custom stop words), lemmatized and lowercased everything, then combined it all into one column. All of these steps can be seen in the code below which is taken from my utils.py file.

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from gensim import corpora
from gensim.utils import simple_preprocess
import pandas as pd

def preprocess_text_column(text):
    """
    Preprocesses a text column for LDA analysis.
    """
    stop_words = set(stopwords.words('english'))
    # adding days of the week to stop words
    stop_words.update(['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday'])
    # adding months to stop words
    stop_words.update(['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august',
                       'september', 'october', 'november', 'december'])
    lemmatizer = WordNetLemmatizer()
    
    try: 
        words = word_tokenize(text.lower())
        words = [w for w in words if not w in stop_words]
        words = [lemmatizer.lemmatize(w) for w in words]
        words = simple_preprocess(str(words), deacc=True)

        return ' '.join(words)
    except:
        return text
    
def clean_raw_webscraper_data(fname):
    """
    Takes in a file of raw avalanche reports data from the webscraper, and returns a dataframe of the cleaned
    """
    # Reading in the CSV file:
    all_data = pd.read_csv(fname)
    # Adding Column Names:
    all_data.columns = ['date', 'zone', 'overall_risk', 'above_treeline_risk', 'near_treeline_risk', 'below_treeline_risk', 'bottom_line_text', 'problem_type_text', 'forecast_discussion_text']
    # Adding a column for the combined text of all 3 text columns:
    all_data['combined_text'] = all_data['bottom_line_text'] + all_data['problem_type_text'] + all_data['forecast_discussion_text']
    # Converting date column to datetime:
    all_data['date'] = pd.to_datetime(all_data['date'])
    
    # Processing all text columns:
    text_coloumns = ['bottom_line_text', 'problem_type_text', 'forecast_discussion_text', 'combined_text']
    
    for column in text_coloumns:
        all_data[column] = all_data[column].apply(preprocess_text_column)

    all_data['zone'] = all_data['zone'].str.lower()
    all_data['overall_risk'] = all_data['overall_risk'].str.lower()

    # Mapping risk ratings to numbers:
    rating_mapping = {
        "extreme": 5.0,
        "high": 4.0,
        "considerable": 3.0,
        "moderate": 2.0,
        "low": 1.0,
        "no rating": 0.0
    }

    all_data['overall_risk'] = all_data['overall_risk'].map(rating_mapping)

    return all_data

### Step 3: Making Predictions and Using LLM's

My intiail plan for this project involved using an LSTM or some similar model to predict the avalanche risk for the next week. While I 

In [30]:
import keys
from langchain import VectorDBQA, OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.document_loaders import DataFrameLoader
import pandas as pd
import pinecone

api_key = keys.OPENAI_KEY

# Loading the data:
all_data = pd.read_csv('output_data/all_zones_all_data.csv')
all_data = all_data.drop(columns=['bottom_line_text', 'problem_type_text', 'forecast_discussion_text'])
all_data = all_data.dropna()
all_data.head()

,date,zone,overall_risk,above_treeline_risk,near_treeline_risk,below_treeline_risk,combined_text,tavg,tmin,tmax,prcp,wdir,pres,tsun
0,2022-04-23,east slopes north,3.0,3.0,3.0,2.0,avalanche danger minimal morning quickly spike...,7.6,-0.9,15.9,0.0,225.3,1024.0,0.0
1,2022-04-23,west slopes central,3.0,3.0,3.0,2.0,avalanche danger minimal morning quickly spike...,9.9,4.6,15.4,0.0,242.4,1026.2,0.0
2,2022-04-23,west slopes south,3.0,3.0,3.0,2.0,weather look beautiful could create dangerous ...,9.9,4.4,16.3,0.0,279.4,1026.0,0.0
3,2022-04-23,east slopes central,2.0,2.0,2.0,2.0,spring shed underway multi day warming trend p...,10.6,5.3,16.3,0.0,282.1,1025.4,0.0
4,2022-04-23,snoqualmie pass,3.0,3.0,3.0,2.0,weather look beautiful however sunshine warm t...,10.2,4.8,16.3,0.0,274.1,1025.8,0.0


In [31]:

# Loading the data:
loader = DataFrameLoader(all_data, 'combined_text')
documents = loader.load()
# Initializing
# Splitting the data:
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=10)
docs = text_splitter.split_documents(documents)

In [32]:

# Getting the embeddings:
embeddings = OpenAIEmbeddings(openai_api_key=api_key)

pinecone.init(api_key=keys.PINECONE_KEY, environment='us-west1-gcp-free')


In [39]:
# Using Pinecone as the vectorstore:
index_name = "avalanche-reports"
docsearch = Pinecone.from_documents(docs, embeddings, index_name=index_name)


In [44]:
query = "Warm wind from the southeast"
results = docsearch.similarity_search(query)
print(results)

[Document(page_content="Periods of moderate E-SE winds will continue to move snow at upper elevations.  While most wind slabs will be shallow, larger avalanches will become possible the longer this pattern continues.  As light new snow continues to accumulate in wind-sheltered areas, the chances of natural or human-triggered dry loose avalanches in very steep terrain will increase. We'll see E-SE ridgeline winds generally ramp up and move an increasing about of loose snow near and especially above treeline tonight through Thursday night, gradually increasing the likelihood that you'll find and trigger a wind slab avalanche.  Use caution on lee steep slopes below ridges, steering around convex rolls and avoiding steep terrain below the base of cliffs.  You can feel for stiff over soft snow layering and look for signs of cracking to identify areas of potentially unstable, wind-drifted snow.  Cornices that formed during recent periods of strong W-SW winds may not help you identify which s

In [43]:
llm = OpenAI(openai_api_key=api_key)
qa = VectorDBQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    vectorstore=docsearch,
    return_source_documents=False
)
query = "What weather conditions lead to the highest avalanche rating?"
results = qa.run(query)
print(results)

 Dangerous avalanche conditions are expected on Monday as temperatures rise and rain continues to fall. Increasing freezing levels and precipitation in the form of rain will set the stage for wet snow avalanches.
